In [29]:
# Custom Modules import
from util.data_loader import load_competitions, load_matches
from util.league_table import generate_league_table
from util.elo_calculator import initialize_team_stats, calculate_elo_ratings, loss_from_comparing_tables, elo_grid_search

from statsbombpy import sb
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Nuanced Team Strength Model

### Used Features to quantify Team Stregnth:
- Possession
- Shots on Goal/on Target
- Passes 
- Duels
- 


In [30]:
events = sb.events(match_id=3890259)
events.shape

/Users/mpr/anaconda3/envs/BDEProject/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


(3691, 91)

In [31]:
distinct_types = events['type'].unique()
print(distinct_types)

['Starting XI' 'Half Start' 'Pass' 'Pressure' 'Ball Receipt*' 'Carry'
 'Duel' 'Clearance' 'Ball Recovery' 'Interception' 'Dispossessed'
 'Miscontrol' 'Block' 'Foul Committed' 'Foul Won' 'Shot' 'Goal Keeper'
 'Shield' '50/50' 'Dribble' 'Dribbled Past' 'Injury Stoppage' 'Player Off'
 'Player On' 'Half End' 'Error' 'Substitution' 'Tactical Shift']


In [32]:
print(events['pass_outcome'].unique())
print(events['pass_type'].unique())
print(events['pass_height'].unique())

[nan 'Incomplete' 'Out' 'Unknown' 'Pass Offside']
[nan 'Kick Off' 'Throw-in' 'Recovery' 'Free Kick' 'Goal Kick' 'Corner'
 'Interception']
[nan 'Ground Pass' 'High Pass' 'Low Pass']


In [41]:
won_duels = events[
    (events['type'] == 'Duel') |
     #(events['duel_outcome'] == 'Success In Play') |
    (events['type'] == '50/50')
].dropna(axis=1, how='all')

won_duels

,50_50,counterpress,duel_outcome,duel_type,duration,id,index,location,match_id,minute,...,possession,possession_team,possession_team_id,related_events,second,team,team_id,timestamp,type,under_pressure
3229,NaN,NaN,NaN,Aerial Lost,0.0,7cedaf2b-42f2-4c6b-8b39-d31d4c933eee,15,"[88.4, 46.3]",3890259,0,...,2,Hamburger SV,171,[63dd449c-ec9a-4bc6-9fc4-c6c088a08e0d],8,Hamburger SV,171,00:00:08.373,Duel,True
3230,NaN,NaN,Lost Out,Tackle,0.0,6a3009c1-7ac2-4ac4-86cd-8179007059bd,32,"[37.1, 73.3]",3890259,0,...,3,Bayern Munich,169,[e7896a57-20f2-4e6e-9f70-d6be16d2e3b5],17,Hamburger SV,171,00:00:17.986,Duel,True
3231,NaN,True,Lost In Play,Tackle,0.0,a19abfb5-682e-40af-899d-5f0e8e9440d3,135,"[79.2, 75.1]",3890259,1,...,6,Hamburger SV,171,[dbb3c1ad-962d-4c55-a631-ddf12758458c],59,Bayern Munich,169,00:01:59.230,Duel,True
3232,NaN,NaN,NaN,Aerial Lost,0.0,682d87dd-1887-4fa3-a496-0309bf0c812f,180,"[82.5, 50.1]",3890259,3,...,9,Hamburger SV,171,[c99ff0ee-6758-4428-870f-acb3432ea23c],17,Hamburger SV,171,00:03:17.740,Duel,True
3233,NaN,NaN,NaN,Aerial Lost,0.0,3096632c-2721-403c-a1c2-9bd0dce2490d,243,"[81.6, 25.0]",3890259,4,...,13,Bayern Munich,169,[5364aff2-256c-4135-a155-578835ece3b4],20,Bayern Munich,169,00:04:20.679,Duel,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3302,NaN,NaN,NaN,Aerial Lost,0.0,5a63857c-4670-4ac4-a51b-611c0e712794,3484,"[77.9, 30.9]",3890259,84,...,161,Hamburger SV,171,[b1d63214-a41a-4468-9077-8f7b5379db2c],55,Hamburger SV,171,00:39:55.694,Duel,True
3303,NaN,NaN,Lost In Play,Tackle,0.0,45df94c6-e4bf-4993-a129-c4ade35d0b4c,3493,"[46.0, 58.2]",3890259,85,...,161,Hamburger SV,171,[41e442db-9b78-478d-abab-903f355467b2],0,Bayern Munich,169,00:40:00.770,Duel,True
3304,NaN,True,Success In Play,Tackle,0.0,fec7a660-4396-4808-bb33-7ca56b8606be,3540,"[109.0, 57.0]",3890259,86,...,163,Bayern Munich,169,[33774d49-6164-442b-8760-c28126e7b6d5],13,Bayern Munich,169,00:41:13.188,Duel,True
3628,"{'outcome': {'id': 1, 'name': 'Lost'}}",NaN,NaN,NaN,0.0,891e62dc-e747-442f-959f-106719f11f4f,290,"[76.9, 1.0]",3890259,5,...,14,Hamburger SV,171,[8e60bce3-31de-4bee-8944-36fcb7bf0572],22,Hamburger SV,171,00:05:22.621,50/50,True


In [42]:
won_duels_grouped = events[
    (events['type'] == 'Duel') &
    (events['duel_outcome'] == 'Success In Play')
].dropna(axis=1, how='all').groupby('team').size().reset_index(name='duels_won')

won_duels_grouped

,team,duels_won
0,Bayern Munich,7
1,Hamburger SV,7


In [35]:
# Calculate completed passes (successful passes with no outcome)
completed_passes = (events[(events['type'] == 'Pass') & 
                         (events['pass_outcome'].isnull())]
                  .groupby('team')
                  .size()
                  .reset_index(name='completed_passes'))

# Calculate total passes
total_passes = (events[events['type'] == 'Pass']
               .groupby('team')
               .size()
               .reset_index(name='total_passes'))

# Merge the two DataFrames
pass_stats = pd.merge(completed_passes, total_passes, on='team')

# Calculate completion percentage
pass_stats['completion_percentage'] = (pass_stats['completed_passes'] / 
                                     pass_stats['total_passes']) * 100

# Sort by completion percentage
pass_stats = pass_stats.sort_values('completion_percentage', ascending=False)

pass_stats.head()

,team,completed_passes,total_passes,completion_percentage
0,Bayern Munich,711,797,89.209536
1,Hamburger SV,150,240,62.500000
